In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np

from keras.datasets import mnist
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")
import os
os.environ["KMP_SETTINGS"] = "false"

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# loading the dataset into the appropriate variables
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
print('Size of the train_images tensor: ', train_images.shape) # 60000 images of 28x28 resolution
print('Size of the train_labels tensor: ', train_labels.shape) # 60000 labels
print('Size of the test_images tensor: ', test_images.shape) # 10000 images of 28x28 resolution
print('Size of the test_labels tensor: ', test_labels.shape) # 10000 labels

Size of the train_images tensor:  (60000, 28, 28)
Size of the train_labels tensor:  (60000,)
Size of the test_images tensor:  (10000, 28, 28)
Size of the test_labels tensor:  (10000,)


In [4]:
# the input data is reshaped i.e. the 2d image 28x28 matrix is flattened into a a single 784 array
# after flattening the values of the array are divided by 255 to normalize their values between 0 & 1 
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [5]:
# one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [6]:
# Sequential model gives us the opportunity to create a linear model 
network = models.Sequential()

# the first dense layer has 512 nodes and a relu activation function 
# the input function is equal to the resolution of the images
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))

# the second dense layer is a 10-way softmax layer which gives 10 probability scores which all add up to 1, each score will be the probability that the image represents one of the 10 digit classes
network.add(layers.Dense(10, activation='softmax'))
# summary of the network architecture 
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2022-01-05 13:18:03.906058: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
# the loss function basically measures the performance of the model on the training data
# Use this crossentropy loss function when there are two or more label classes
network.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [8]:
# passing the train images to train the network 
history = network.fit(train_images, train_labels, epochs=5, batch_size=128)

2022-01-05 13:18:04.319768: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
469/469 [==============================] - 4s 7ms/step - loss: 0.2587 - accuracy: 0.9253
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1032 - accuracy: 0.9696
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0667 - accuracy: 0.9801
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0486 - accuracy: 0.9854
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0367 - accuracy: 0.9894


In [9]:
# calculating validation loss and accuracy
val_loss, val_acc = network.evaluate(test_images, test_labels)
print('Validation loss:', val_loss, '\nValidation accuracy: ', val_acc)
loss = history.history['loss']
acc = history.history['accuracy']

313/313 [==============================] - 1s 2ms/step - loss: 0.0622 - accuracy: 0.9809
Validation loss: 0.06217947229743004 
Validation accuracy:  0.98089998960495


In [10]:
# plotting the training loss and accuracy 
fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Loss',  'Training Accuracy'))

fig.add_trace(go.Scatter(name='Loss', 
                         x=[1, 2, 3, 4, 5], 
                         y=[loss[0], loss[1], loss[2], loss[3], loss[4]]),
                         row=1, 
                         col=1)

fig.add_trace(go.Scatter(name='Accuracy', 
                         x=[1, 2, 3, 4, 5], 
                         y=[acc[0], acc[1], acc[2], acc[3], acc[4]]),
                         row=1, 
                         col=2)

fig.update_layout(title_text='ReLu and Softmax as Activation functions')
fig.show()

In [11]:
lst = []
layer_1 = []
layer_2 = []
val_loss_2 = []
val_acc_2 = []
def filing(activation_1, activation_2, val_loss_1, val_acc_1):
    lst.append(activation_1+activation_2)
    layer_1.append(activation_1)
    layer_2.append(activation_2)
    val_loss_2.append(val_loss_1)
    val_acc_2.append(val_acc_1)
def model_train(activation_1, activation_2):
    network = models.Sequential()
    network.add(layers.Dense(512, activation=activation_1, input_shape=(28 * 28,)))
    network.add(layers.Dense(10, activation=activation_2))
    network.summary()
    print()
    
    network.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    history = network.fit(train_images, train_labels, epochs=5, batch_size=128)
    print()
    
    val_loss_1, val_acc_1 = network.evaluate(test_images, test_labels)
    print('Validation loss:', val_loss_1, '\nValidation accuracy: ', val_acc_1)
    loss_1 = history.history['loss']
    acc_1 = history.history['accuracy']
    print()
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Loss',  'Training Accuracy'))

    fig.add_trace(
        go.Scatter(name='Loss', 
               x=[1, 2, 3, 4, 5], 
               y=[loss_1[0], loss_1[1], loss_1[2], loss_1[3], loss_1[4]]),
               row=1, 
               col=1)

    fig.add_trace(
        go.Scatter(name='Accuracy', 
               x=[1, 2, 3, 4, 5], 
               y=[acc_1[0], acc_1[1], acc_1[2], acc_1[3], acc_1[4]]),
               row=1, 
               col=2)
    text = 'Activation function: '+activation_1+' '+activation_2
    fig.update_layout(title_text=text)
    fig.show()
    
    filing(activation_1, activation_2, val_loss_1, val_acc_1)

In [12]:
activation_functions = ['elu', 'gelu', 'relu', 'selu', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential', 'softmax', 'softplus', 'softsign', 'swish', 'tanh']
for i in activation_functions:
    for j in activation_functions:
        model_train(i,j)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 9.6651 - accuracy: 0.3113
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 7.8405 - accuracy: 0.4270
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 6.7813 - accuracy: 0.4494
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 8.3060 - accuracy: 0.4159
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 9.0494 - accuracy: 0.3722

313/313 [====

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 4s 8ms/step - loss: 1.6816 - accuracy: 0.7579
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 1.4148 - accuracy: 0.8216
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 1.3973 - accuracy: 0.8403
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 1.5247 - accuracy: 0.8344
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 1.7524 - accuracy: 0.8269

313/313 [====

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0988
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0987
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0987
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0987
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0987

313/313 [===================

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 4s 8ms/step - loss: 9.1099 - accuracy: 0.3291
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 8.4154 - accuracy: 0.3279
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 7.7736 - accuracy: 0.3711
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 8.2072 - accuracy: 0.3566
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 7.6650 - accuracy: 0.3946

313/313 [====

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3383 - accuracy: 0.9003
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1757 - accuracy: 0.9479
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1181 - accuracy: 0.9645
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0882 - accuracy: 0.9738
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0699 - accuracy: 0.9788

313/313 [====

...

Model: "sequential_165"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_330 (Dense)            (None, 512)               401920    
_________________________________________________________________
dense_331 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3362 - accuracy: 0.9005
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1740 - accuracy: 0.9492
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1166 - accuracy: 0.9655
Epoch 4/5
469/469 [==============================] - 3s 5ms/step - loss: 0.0852 - accuracy: 0.9744
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0648 - accuracy: 0.9806

313/313 [==

Model: "sequential_166"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_332 (Dense)            (None, 512)               401920    
_________________________________________________________________
dense_333 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3235 - accuracy: 0.9072
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1515 - accuracy: 0.9556
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1024 - accuracy: 0.9696
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0753 - accuracy: 0.9775
Epoch 5/5
469/469 [==============================] - 4s 7ms/step - loss: 0.0578 - accuracy: 0.9830

313/313 [==

Model: "sequential_167"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_334 (Dense)            (None, 512)               401920    
_________________________________________________________________
dense_335 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 8.5819 - accuracy: 0.1557
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 7.2456 - accuracy: 0.1465
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 7.7600 - accuracy: 0.1336
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 7.8826 - accuracy: 0.1022
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 7.8895 - accuracy: 0.1022

313/313 [==

Model: "sequential_168"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_336 (Dense)            (None, 512)               401920    
_________________________________________________________________
dense_337 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 3.4164 - accuracy: 0.6466
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 2.6640 - accuracy: 0.7551
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 2.8050 - accuracy: 0.7255
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 3.3630 - accuracy: 0.6971
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 3.8350 - accuracy: 0.6834

313/313 [==

Model: "sequential_169"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_338 (Dense)            (None, 512)               401920    
_________________________________________________________________
dense_339 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 9.6201 - accuracy: 0.1717
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 9.6537 - accuracy: 0.2040
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.1051
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0987
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: nan - accuracy: 0.0987

313/313 [===========